# Process Wavfiles

```
Here, we will extract acoustic features from wavefiles.

Notes
-----
1. This notebook takes ~2 hours to run for a single subject (16 GB RAM, Intel i5 11th Gen Processor)
2. Manually update 'subject' variable to run for chosen subject (5 in total).
```

In [1]:
# Import math, plotting and sound libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from soundsig.sound import BioSound, WavFile
import os

# Make plots on this Notebook.
%matplotlib inline

In [2]:
df = pd.read_csv('aa-4.csv')
df.sample(5)

,subject,site,single_unit,sortType,stim_number,original_wavfile,callerAge,callid,stim_duration,stim_source,stim_source_sex,stim_type,brain_region,hemisphere,vocid,auditory_cortex
18082,GreBlu9508M,Site1,Site1_L750R1100_e21_s0_ss1,single,52,BlaBla0506_MAF_Te_3-5-10.wav,A,Te,2.500000,familiar,m,call,CMM,R,Te,A2
76529,YelBlu6903F,Site1,Site1_L500R400_e25_s0_ss2,single,140,BluRas61dd_MAS_Ag_3.wav,A,Ag,0.168469,unfamiliar,m,call,CMM,R,Ag,A2
100538,YelBlu6903F,Site4,Site4_L1500R1500_e30_s0_ss4,single,99,WhiWhi1415_FAF_DC_6-2-4.wav,A,DC,2.500000,familiar,f,call,NCM,R,DC,A2
11026,BlaBro09xxF,Site3,Site3_L2500R2300_e1_s1_ss1,single,86,STRFxx0000__So_19.wav,NaN,NaN,2.445032,unfamiliar,m,song,NCM,L,song,A2
55509,WhiWhi4522M,Site2,Site2_L1500R1450_e13_s0_ss3,single,133,BluRas07dd_FAS_Te_2-7-9.wav,A,Te,2.500000,unfamiliar,f,call,NCM,L,Te,A2


In [16]:
df['stim_number'][0]

100

In [3]:
subject = 'WhiWhi4522M'

In [12]:
base_path = os.getcwd()

# Go to the folder that has the wav files
data_root = 'C:\\Users\\Dell\\Desktop\\avian-audition\\data\\anesthetized'
wave_root = os.path.join(data_root, subject, 'wavfiles')

# This will be the output directory
if not os.path.exists(os.path.join(wave_root, 'h5files')):
    os.makedirs(os.path.join(wave_root, 'h5files'))

In [25]:
df[(df['subject']==subject)&(df['stim_number']==int(1))]['vocid']

Series([], Name: vocid, dtype: object)

In [23]:
stim_number

'1'

In [26]:
os.chdir(wave_root)

# Plot flag to decide whether or not to plot output
plotMe = False
normalize = False

# Find all the wave files 
isound = 0   
for fname in os.listdir(wave_root):
    if fname.endswith('.wav'):
        isound += 1
        
        # Read the sound file
        print ('Processing sound %d:%s\n' % (isound, fname))
        
        # Mono == True forces the sound stored to be a single channel as bioSound does not deal with stereo.
        soundIn = WavFile(file_name=fname, mono=True) 
        filename, file_extension = os.path.splitext(fname)        
        
        # Here we parse the filename to get the birdname and the call type. 
        # You will have to write custom code to extract your own identifiers.
        stim_number = filename[filename.find('stim')+4:]
        
        birdname = subject
        calltype = df[(df['subject']==subject)&(df['stim_number']==int(stim_number))]['vocid'].unique()
        if len(calltype) == 0:
            continue
        assert len(calltype) == 1
        calltype = calltype[0]
        
        # Normalize if wanted
        if normalize :
            maxAmp = np.abs(soundIn.data).max() 
        else :
            maxAmp = 1.0
    
    # Create BioSound Object
        myBioSound = BioSound(soundWave=soundIn.data.astype(float)/maxAmp, fs=float(soundIn.sample_rate), emitter=birdname, calltype = calltype)
             
    # Calculate the spectrogram and the rms
        myBioSound.spectroCalc(spec_sample_rate=1000, freq_spacing = 50, min_freq=0, max_freq=10000)
        myBioSound.rms = myBioSound.sound.std() 
       
    # Calculate amplitude enveloppe
        myBioSound.ampenv(cutoff_freq = 20, amp_sample_rate = 1000)
       
    # Calculate the power spectrum
        myBioSound.spectrum(f_high=10000)
    
    # Calculate fundamental and related values.  These are the default values.
    # For the estimation of the fundamental, four methods are available: 
    # 'AC' - Peak of the auto-correlation function
    # 'ACA' - Peak of envelope of auto-correlation function 
    # 'Cep' - First peak in cepstrum 
    # 'Stack' - Fitting of harmonic stacks (default - works well for zebra finches)
    
        myBioSound.fundest(maxFund = 1500, minFund = 300, lowFc = 200, highFc = 6000, 
                           minSaliency = 0.5, debugFig = 0, 
                           minFormantFreq = 500, maxFormantBW = 500, windowFormant = 0.1,
                           method='Stack')
      
    # Calculate the MPS
        myBioSound.mpsCalc(window=0.1, Norm = True)
                
        if plotMe and isound <= 10: 
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            print('                Bird %s    Call Type %s' % (myBioSound.emitter, myBioSound.type))
            myBioSound.plot(DBNOISE=50, f_low=250, f_high=10000)  
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

    # Save the results
        fh5name = 'h5files/%s.h5' % (filename)
        myBioSound.saveh5(fh5name)
        
os.chdir(base_path)
print('Done')

Processing sound 1:stim1.wav

Processing sound 2:stim10.wav

Processing sound 3:stim100.wav



C:\Users\Dell\Desktop\avian-audition\analysis\anesthetized\soundsig\sound.py:285: RuntimeWarning: divide by zero encountered in log10
  self.spectro = 20*np.log10(np.abs(spec))


Processing sound 4:stim101.wav



C:\Users\Dell\Desktop\avian-audition\analysis\anesthetized\soundsig\sound.py:285: RuntimeWarning: divide by zero encountered in log10
  self.spectro = 20*np.log10(np.abs(spec))


Processing sound 5:stim102.wav



...

Processing sound 287:stim97.wav



C:\Users\Dell\Desktop\avian-audition\analysis\anesthetized\soundsig\sound.py:285: RuntimeWarning: divide by zero encountered in log10
  self.spectro = 20*np.log10(np.abs(spec))


Processing sound 288:stim98.wav



C:\Users\Dell\Desktop\avian-audition\analysis\anesthetized\soundsig\sound.py:285: RuntimeWarning: divide by zero encountered in log10
  self.spectro = 20*np.log10(np.abs(spec))


Processing sound 289:stim99.wav



In [28]:
print('Done')

Done
